# Time Dependent Variational Principle (TDVP) and large time steps
TDVP as used in the FTPS solver for impurity models has the peculiar property that it allows for very large time steps $\Delta t$ and results can even get worse when decreasing $\Delta t$ without reducing the truncated weight at the same time. <br>
This is in contrast to the other commonly employed time evolution method the Time Evolving Block Decimation (TEBD). At this point it is not important how any of these two algorithms work, one only needs to know that these are two very different methods of performing tensor network time evolution and the error of both should scale $~\Delta t^2$.<br>

In this tutorial, we will calculate the Green's function for impurity models using various time step sizes and in each case by employing TEBD and TDVP.

In [ ]:
#%matplotlib notebook
import matplotlib.pyplot as plt

import forktps as ftps
from forktps.DiscreteBath import *
from forktps.Helpers import *
from forktps.solver import DMRGParams, TevoParams

import numpy as np

from triqs.gf import *
from h5 import *

The first few steps are the same as in the previous tutorials, define a hybridization, discretize it and look at the reconstruction, define the rest of the model, i.e, the interaction and the on-site energy, and initialize the solver.

In [ ]:
#get hybridization
Norbs = 1
gfstruct = [ ['up',[0]], ['dn',[0]] ]

dup = GfReFreq(indices = [0], window = (-3., 3.), n_points =  3001, name = 'up')
ddn = GfReFreq(indices = [0], window = (-3., 3.), n_points =  3001, name = 'dn')

dup['0','0'] << SemiCircular(half_bandwidth = 2.0)
ddn['0','0'] << SemiCircular(half_bandwidth = 2.0)


Delta = BlockGf(name_list = ('up','dn'), block_list = (dup,ddn), make_copies = False)

#discretize bath and look at the reconstruction
Nb = 29 #number of bath sites
bath = DiscretizeBath(Delta=Delta, Nb=Nb) #bath object storing all relevant information for the bath
eta = 0.1

DeltaDiscrete = bath.reconstructDelta( w = ddn.mesh, eta = eta )

indx = 1
plt.figure(figsize=[8,3])
for name, g in Delta:
    plt.subplot(1,2,indx)
    plt.title(name)
    for j in range(g.target_shape[0]):
        plt.plot( getX(Delta.mesh), -Delta[name][j,j].imag.data, label = 'Original')
        plt.plot( getX(Delta.mesh), -DeltaDiscrete[name][j,j].imag.data, label = 'Reconstruction')
    indx +=1

        
# h0loc and hint
e0 = ftps.solver_core.Hloc(gfstruct) #give the local Hamiltonian the right block structure
e0.Fill([[0]]) # put the same 1x1 matrix on both blocks

Hint = ftps.solver_core.HInt(u=0.,j=0., up=0., dd = False)


#finally initialize the solver
S = ftps.Solver(gf_struct = gfstruct , nw = 3001, wmin=-3., wmax=3.)
S.b = bath
S.e0 = e0

In [ ]:
# note this cell takes some time to execute. Therefore, the execution part is commented out and the tutorial folder
# contains the h5-file with the Green's functions. If you want to execute the code yourself, grab a coffee ...

fname = 'Tutorial_TDVP_LargeTimeSteps.h5'

# note that time steps with size 1.0 are usually pretty crayz
dts =[1.0, 0.5, 0.2, 0.05, 0.02, 0.005]
methods = ['TDVP', 'TEBD']


## Use same truncated weight for all time steps and for DMRG as well as time evolution
#tw = 1E-8
#
## loop over dt and method
#for method in methods:
#    for dt in dts:
#        print('Doing dt =', dt, method)
#        NTimeSteps = int( round(2./dt) )
#        
#        dmrg = DMRGParams(tw=tw) # set tw on all links to the same value
#        tevo = TevoParams(tw=tw, dt=dt, time_steps=NTimeSteps, method = method) # set tw on all links to the same value
#
#        S.solve( h_int = Hint, 
#                 tevo=tevo,
#                 params_GS = dmrg,
#                 eta = eta, 
#                 calc_me = [['up', 0]] # only calculate the Green's function of block 'up' with index 0
#                 )
#    
#        with HDFArchive(fname, 'a') as a:
#            a[f'Ggr_{method}_{dt:.3f}'] = S.G_gr
        

# read in data
Ggrs = {}
for method in methods:
    Ggrs[method] = {}
    for dt in dts:
        with HDFArchive(fname, 'r') as a:
            Ggrs[method][dt] = a[f'Ggr_{method}_{dt:.3f}']
 

In [ ]:
Ggr_exact = CalcGgr(bath=bath, hloc=e0, mesh = Ggrs[methods[-1]][dts[-1]].mesh )
t = getX(Ggr_exact.mesh)


# plot numerical error
maxError = {}
plt.figure(figsize=(8, 4))
for indx, method in enumerate( methods ):
    plt.subplot(1,2,indx+1)
    plt.title(f'Error of $G^>(t)$ using {method}')
    maxError[method] = []
    for dt in dts:
        Ggr = Ggrs[method][dt]    
        Ggr_exact = CalcGgr(bath=bath, hloc=e0, mesh = Ggr.mesh )


        t = getX(Ggr.mesh)
        y = np.abs(Ggr['up'].data[:,0,0] - Ggr_exact['up'].data[:,0,0])
        maxError[method].append( np.max(y) )

        plt.semilogy( t, y, label = f'dt = {dt:.3}')
    plt.ylim(1E-8, 1E-1)
    plt.xlabel('t')
    plt.ylabel('$|G_{ex} - G_{FTPS}|$')
    plt.legend()

    
# plot maximum of error
plt.figure(figsize = [5,4])
plt.title('Maximum Error')
for method in methods:
    plt.loglog(dts, maxError[method], 'x-' ,label = method )
plt.loglog(dts, np.array(dts)**2*0.02, '--' , label = '~$\Delta t^2$' , color= 'black' )
plt.ylabel('max($G_{ex} - G_{FTPS}$)')
plt.xlabel('$\Delta t$')
plt.legend()

Starting with TEBD: At large time steps, its error decreases and then seems to saturate and even grow again slightly. From $\Delta t = 0.05$ upwards it shows near perfect $~\Delta t^2$ scaling so everything is as expected.

TDVP on the other hand is very different. Its minimum error is at $\Delta t = 0.2$ and it gives quite accurate results for huge time steps up to $\Delta t = 1.0$. Below $0.2$ the error actually increase drastically and at the smallest time step they are of order $0.1$. A scaling $~\Delta t^2$ is maybe starts at the highest two values but is not visible for a wide range of time steps.

It is important to note that this is because of the interplay between time step and truncation. A lower value of $\Delta dt$ still allows to find the Green's function with higher precision, but therefore one has to decrease the truncation as well! And as it turns out that this seems to be a much bigger concern for TDVP than for TEBD.

So keep in mind to not use a very small time steps when using the default method TDVP and check the calculation for convergence also with a LARGER value of $\Delta t$!